In [1]:
import pickle

fname = '../../data/fmri-FC-slim.pkl'
fmriDict = None

with open(fname, 'rb') as f:
    fmriDict = pickle.load(f)
    
print(fmriDict.keys())

dict_keys(['FC-slim', 'subjNum2IdxMap', 'subjIdx2NumMap', 'groupsNormalDiagMap'])


In [8]:
# Package fMRI data into data matrix and response variables

import numpy as np

keys = list(fmriDict['groupsNormalDiagMap'].keys())
y = [fmriDict['groupsNormalDiagMap'][key] for key in keys]
y = np.array(y).astype('int')
X = [fmriDict['FC-slim'][fmriDict['subjNum2IdxMap'][key]] for key in keys]
X = np.stack(X)
print(X.shape)
print(y.shape)

(66, 34716)
(66,)


In [19]:
import random
import numpy as np

from sklearn.linear_model import LogisticRegression
import sys

sys.path.append('../..')

from imagenomer.main import Analysis, JsonData
from imagenomer.metadata import get_power_community_metadata

# Create ROI labels
a,b = np.triu_indices(264,1)
idcs = np.arange(34716)

labels = [f'{a[i]}-{b[i]}' for i in idcs]

print(labels[0:10])
print(labels[-10:])

nRuns = 50
C = 1e-2
max_iter = 5000
modelName = 'Logistic Regression'

# Start analysis
model = f'{modelName} L2 C={C} max_iter={max_iter}'
desc = 'normal(1) vs. fibromyalgia(0) rest fMRI only W only (standardization)'
analysis = Analysis(desc=f'{desc}: {model}', 
                    host='localhost', 
                    port=8000, 
                    label_names=['Flattened Indices', 'ROIs'],
                    labels=[list(range(X.shape[-1])), labels])

# Send community metadata
meta = get_power_community_metadata(analysis)
r = meta.post()
print(r.content)

tot = []

for run in range(nRuns):
    FCslim = fmriDict['FC-slim']
    subjNum2Idx = fmriDict['subjNum2IdxMap']

    # Make a random split
    idcs = np.arange(66)
    np.random.shuffle(idcs)
    trainIdcs = idcs[:50]
    testIdcs = idcs[50:]
    Xtr = X[trainIdcs]
    Xt = X[testIdcs]
    ytr = y[trainIdcs]
    yt = y[testIdcs]
    
    # Standardize - good for some datasets but not for others
#     mu = np.mean(Xtr, axis=0, keepdims=True)
#     sd = np.std(Xtr, axis=0, keepdims=True)
#     Xtr = (Xtr-mu)/sd
#     Xt = (Xt-mu)/sd
    
    # HC and FM subjects
    HCtrain = np.where(ytr == 1)[0]
    FMtrain = np.where(ytr == 0)[0]
    HCtest = Xtr.shape[0] + np.where(yt == 1)[0]
    FMtest = Xtr.shape[0] + np.where(yt == 0)[0]

    clf = LogisticRegression(max_iter=5000, C=C).fit(Xtr, ytr)
#     clf = LogisticRegression(max_iter=5000, C=0.001, 
#                              solver='saga', penalty='l1').fit(Xtrain, ytrain)
    yhat = clf.predict(Xt)
    acc = sum(yhat == yt)/len(yt)
    
    print(f'{run}. {acc}')
    
    tot.append(acc)
    
    # Raw weights
    w = list(clf.coef_.squeeze())
    
    # Send Weights
    jsonObj = {
        'Compare': desc,
        'Model': modelName,
        'Accuracy': float(acc),
        'Train': [HCtrain.shape[0],FMtrain.shape[0]],
        'Test': [33-HCtrain.shape[0],33-FMtrain.shape[0]],
        'Weights': w
    }
    
    dat = JsonData(analysis)
    dat.update(jsonObj)
    r = dat.post()
    print(dat.dict['runid'])
    print(r.content)
    
tot = np.array(tot)
print(np.mean(tot))
print(np.std(tot))

['0-1', '0-2', '0-3', '0-4', '0-5', '0-6', '0-7', '0-8', '0-9', '0-10']
['259-260', '259-261', '259-262', '259-263', '260-261', '260-262', '260-263', '261-262', '261-263', '262-263']
b'Success'
0. 0.625
0
b'Success'
1. 0.625
1
b'Success'
2. 0.4375
2
b'Success'
3. 0.375
3
b'Success'
4. 0.5625
4
b'Success'
5. 0.6875
5
b'Success'
6. 0.5625
6
b'Success'
7. 0.4375
7
b'Success'
8. 0.625
8
b'Success'
9. 0.5625
9
b'Success'
10. 0.5625
10
b'Success'
11. 0.375
11
b'Success'
12. 0.5625
12
b'Success'
13. 0.4375
13
b'Success'
14. 0.625
14
b'Success'
15. 0.375
15
b'Success'
16. 0.5625
16
b'Success'
17. 0.5625
17
b'Success'
18. 0.4375
18
b'Success'
19. 0.625
19
b'Success'
20. 0.5625
20
b'Success'
21. 0.5
21
b'Success'
22. 0.375
22
b'Success'
23. 0.5625
23
b'Success'
24. 0.5625
24
b'Success'
25. 0.625
25
b'Success'
26. 0.625
26
b'Success'
27. 0.75
27
b'Success'
28. 0.5
28
b'Success'
29. 0.625
29
b'Success'
30. 0.6875
30
b'Success'
31. 0.3125
31
b'Success'
32. 0.5625
32
b'Success'
33. 0.5625
33
b'Succe

In [15]:
commNames = []
commAffil = {}
commAbrev = 'SMH,SMM,CNG,AUD,DMN,MEM,VIS,FRT,SAL,SUB,VTR,DRL,CB,UNK'.split(',')
commCount = np.zeros(14)

powerAffilFname = '../../power/power264CommunityAffiliation.1D'

with open(powerAffilFname, 'r') as f:
    for i,line in enumerate(f.readlines()):
        commAffil[i] = int(line)-1
        commCount[commAffil[i]] += 1
        
print(commAffil)
print(commCount.astype('int'))

{0: 13, 1: 13, 2: 13, 3: 13, 4: 13, 5: 13, 6: 13, 7: 13, 8: 13, 9: 13, 10: 13, 11: 13, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 2, 47: 2, 48: 2, 49: 2, 50: 2, 51: 2, 52: 2, 53: 2, 54: 2, 55: 2, 56: 2, 57: 2, 58: 2, 59: 2, 60: 3, 61: 3, 62: 3, 63: 3, 64: 3, 65: 3, 66: 3, 67: 3, 68: 3, 69: 3, 70: 3, 71: 3, 72: 3, 73: 4, 74: 4, 75: 4, 76: 4, 77: 4, 78: 4, 79: 4, 80: 4, 81: 4, 82: 4, 83: 13, 84: 13, 85: 4, 86: 4, 87: 4, 88: 4, 89: 4, 90: 4, 91: 4, 92: 4, 93: 4, 94: 4, 95: 4, 96: 4, 97: 4, 98: 4, 99: 4, 100: 4, 101: 4, 102: 4, 103: 4, 104: 4, 105: 4, 106: 4, 107: 4, 108: 4, 109: 4, 110: 4, 111: 4, 112: 4, 113: 4, 114: 4, 115: 4, 116: 4, 117: 4, 118: 4, 119: 4, 120: 4, 121: 4, 122: 4, 123: 4, 124: 4, 125: 4, 126: 4, 127: 4, 128: 4, 129: 4, 130: 4, 131: 13, 132: 5, 133: 5, 134: 5, 135: 5, 136: 4

In [16]:
jsonMeta = JsonFCMetadata(analysis)
metaDict = {
    'CommunityMap': commAffil,
    'CommunityNames': commAbrev
}
jsonMeta.update(metaDict)
r = jsonMeta.post()
print(r.content)

b'Success'
